In [ ]:
# Instalación de librerías:

%pip install pandas
%pip install requests
%pip install psycopg2
%pip install datetime
%pip install dotenv

In [378]:
# Importación de librerías a utilizar:

import pandas as pd
import requests as req
import psycopg2 as pg2
from psycopg2.extras import execute_values
import datetime
import linecache

In [379]:
# Configuración para mostrar la totalidad de registros y campos

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [381]:
# Extracción desde la API de los datos diarios de "Los 7 Magníficos" (Alphabet, Amazon, Apple, Meta, Microsoft, Nvidia y Tesla) correspondiente al NASDAQ tomando en cuenta el último año (260 días sin contar Sabados y Domingos):

api = 'https://api.twelvedata.com/time_series?symbol=GOOGL,AMZN,AAPL,META,MSFT,NVDA,TSLA&exchange=NASDAQ&interval=1day&format=JSON&outputsize=260&apikey=' + linecache.getline('datos_seguridad.txt', 1).strip('\n') # Api Key
peticion = req.get(api)
peticion

<Response [200]>

In [382]:
# Conversión de datos a formato JSON:

datos_json = peticion.json()
datos_json

{'GOOGL': {'meta': {'symbol': 'GOOGL',
   'interval': '1day',
   'currency': 'USD',
   'exchange_timezone': 'America/New_York',
   'exchange': 'NASDAQ',
   'mic_code': 'XNGS',
   'type': 'Common Stock'},
  'values': [{'datetime': '2024-02-05',
    'open': '142.82001',
    'high': '145.47000',
    'low': '142.82001',
    'close': '143.67999',
    'volume': '38414428'},
   {'datetime': '2024-02-02',
    'open': '139.25999',
    'high': '142.62000',
    'low': '136.50000',
    'close': '142.38000',
    'volume': '62470600'},
   {'datetime': '2024-02-01',
    'open': '142.12000',
    'high': '143.06000',
    'low': '140.78999',
    'close': '141.16000',
    'volume': '40466500'},
   {'datetime': '2024-01-31',
    'open': '143.62000',
    'high': '144.00000',
    'low': '139.87000',
    'close': '140.10001',
    'volume': '71910000'},
   {'datetime': '2024-01-30',
    'open': '152.80000',
    'high': '153.62000',
    'low': '151.19000',
    'close': '151.46001',
    'volume': '36331800'},
 

In [383]:
# Normalización de datos y agregado de columnas:

nombres = {'GOOGL':'Alphabet Inc.', 'AMZN':'Amazon.com, Inc.', 'AAPL':'Apple Inc.', 'META':'Meta Platforms, Inc.', 'MSFT':'Microsoft Corporation', 'NVDA':'NVIDIA Corporation', 'TSLA':'Tesla, Inc.'}

DataFrame = pd.DataFrame()
claves = list(datos_json.keys())
contador = 0

for empresa in datos_json:
    tabla = pd.json_normalize(datos_json[empresa]['values'])
    tabla['Codigo'] = claves[contador]
    tabla['Empresa'] = nombres[claves[contador]]
    DataFrame = pd.concat([DataFrame, tabla], axis = 0)
    contador += 1

columnas = ['Dia', 'Apertura', 'Maximo', 'Minimo', 'Cierre', 'Volumen', 'Codigo', 'Empresa']
DataFrame.columns = columnas

DataFrame = DataFrame.sort_values(by = ['Dia', 'Empresa'], ascending = [False, True],ignore_index = True)
DataFrame['Id'] = DataFrame.index
DataFrame['Columna_Temporal'] = datetime.datetime.now()

In [401]:
# Conexión a la base de datos en Amazon Redshift:

usuario = linecache.getline('datos_seguridad.txt', 2).strip('\n') # Usuario
contrasena = linecache.getline('datos_seguridad.txt', 3).strip('\n') # Contraseña

anfitrion = linecache.getline('datos_seguridad.txt', 4).strip('\n') # Anfitrion
puerto = linecache.getline('datos_seguridad.txt', 5).strip('\n') # Puerto
base_datos = linecache.getline('datos_seguridad.txt', 6).strip('\n') # Base de Datos

try:
    conexion = pg2.connect(host = anfitrion, port = puerto, dbname = base_datos, user = usuario, password = contrasena)
    print('Conexión exitosa.')
except Exception as e:
    print('Conexión fallida.')
    print(e)

Conexión exitosa.


In [402]:
# Creación de la tabla en Amazon Redshift:

creacion = '''
        DROP TABLE IF EXISTS b_arganaraz_londero_coderhouse.cotizacion_magnificos;

        CREATE TABLE b_arganaraz_londero_coderhouse.cotizacion_magnificos
        (
                Id INT PRIMARY KEY,
                Dia DATE,
                Empresa VARCHAR(50),
                Codigo VARCHAR(50),
                Apertura DECIMAL(10,2),
                Maximo DECIMAL(10,2),
                Minimo DECIMAL(10,2),
                Cierre DECIMAL(10,2),
                Volumen INT,
                Columna_Temporal TIMESTAMP 
        );
        '''

cursor = conexion.cursor()
cursor.execute(creacion)
conexion.commit()

In [403]:
# Insercion de datos a la tabla creada en Amazon Redshift:

insercion = '''
        INSERT INTO b_arganaraz_londero_coderhouse.cotizacion_magnificos(Dia, Apertura, Maximo, Minimo, Cierre, Volumen, Codigo, Empresa, Id, Columna_Temporal)
        VALUES %s    
        '''

valores = [tuple(var) for var in DataFrame.to_numpy()]
execute_values(cursor, insercion, valores)
conexion.commit()

In [404]:
# Seleccion de los datos insertados a la tabla creada en Amazon Redshift y cierre de cursor y conexión:

seleccion = '''
        SELECT * FROM b_arganaraz_londero_coderhouse.cotizacion_magnificos
        '''

cursor.execute(seleccion)
conexion.commit()
resultados = cursor.fetchall()

cursor.close()
conexion.close()

resultados = pd.DataFrame(resultados)
resultados.columns = ['Id', 'Dia', 'Empresa', 'Codigo', 'Apertura', 'Maximo', 'Minimo', 'Cierre', 'Volumen', 'Columna_Temporal']
resultados.set_index('Id', inplace = True)
resultados

,Dia,Empresa,Codigo,Apertura,Maximo,Minimo,Cierre,Volumen,Columna_Temporal
Id,,,,,,,,,
0,2024-02-05,Alphabet Inc.,GOOGL,142.82,145.47,142.82,143.68,38414428,2024-02-05 20:06:36.286325
1,2024-02-05,"Amazon.com, Inc.",AMZN,170.20,170.55,167.93,170.31,54626699,2024-02-05 20:06:36.286325
2,2024-02-05,Apple Inc.,AAPL,188.15,189.25,185.84,187.68,68862037,2024-02-05 20:06:36.286325
3,2024-02-05,"Meta Platforms, Inc.",META,469.88,471.76,459.22,459.41,40417189,2024-02-05 20:06:36.286325
4,2024-02-05,Microsoft Corporation,MSFT,409.90,411.16,403.99,405.65,25019347,2024-02-05 20:06:36.286325
5,2024-02-05,NVIDIA Corporation,NVDA,682.25,694.97,672.05,693.32,66900939,2024-02-05 20:06:36.286325
6,2024-02-05,"Tesla, Inc.",TSLA,184.26,184.67,175.01,181.06,133798212,2024-02-05 20:06:36.286325
7,2024-02-02,Alphabet Inc.,GOOGL,139.26,142.62,136.50,142.38,62470600,2024-02-05 20:06:36.286325
8,2024-02-02,"Amazon.com, Inc.",AMZN,169.19,172.50,167.33,171.81,117154900,2024-02-05 20:06:36.286325
